# Add an Object Store to a Web GIS Deployment

## Introduction

With the ArcGIS Enterprise 11.4 release, the tile cache data store is deprecated. In a future release, it will no longer be supported. To *publish hosted scene layers* or *hosted 3D tiles layers* in an 11.4 ArcGIS Enterprise deployment, you must configure an object store. When you deploy in the Amazon Web Services (AWS) or Microsoft Azure cloud, use a cloud service for the object store. See the [tile cache data store deprecation notice](https://links.esri.com/tile-cache-ds-deprecation) for more information.

This sample notebook demonstrates a workflow for adding an object store in AWS or Azure using the ArcGIS API for Python. The code will work in a Python script as well.

## Import Libraries

In [3]:
from arcgis.gis import GIS

## Connect to your ArcGIS Enterprise Deployment

In [ ]:
gis = GIS(profile="your_enterprise_admin_profile")

> **Note:** See [Storing your credentials locally](/python/latest/guide/working-with-different-authentication-schemes/#storing-your-credentials-locally) for instructions on creating a _profile_.

## List the Federated Servers

When connecting as an admin, the *admin* property returns access to the [PortalAdminManager](python/latest/api-reference/arcgis.gis.admin.html#portaladminmanager), and the [*servers*](/python/latest/api-reference/arcgis.gis.admin.html#arcgis.gis.admin.PortalAdminManager.servers) property on that returns a [ServerManager](/python/latest/api-reference/arcgis.gis.server.html#arcgis.gis.server.ServerManager) object to administer the Enterprise's servers:

In [5]:
server_mgr = gis.admin.servers
server_mgr

< ServerManager @ https://example.site.com/portal/portaladmin >

Use the [*list*](/python/latest/api-reference/arcgis.gis.server.html#arcgis.gis.server.ServerManager.list) method to return all the servers federated with this GIS:

In [6]:
servers = server_mgr.list()
servers

[< Server @ https://example.site.com:6443/arcgis/admin >]

### Retrieve specific Server

Use list indexing to return the specific [server](/python/latest/api-reference/arcgis.gis.server.html#arcgis.gis.server.Server) object to which you'll add the data store:

In [7]:
gis_server =  servers[0]
type(gis_server)

arcgis.gis.server.admin.administration.Server

## Get the Server's Data Store Manager
With the *server* object, the  [*datastores*](/python/latest/api-reference/arcgis.gis.server.html#arcgis.gis.server.Server.datastores) property returns a [DataStoreManager](/python/latest/api-reference/arcgis.gis.server.html#arcgis.gis.server.DataStoreManager) object that provides access to adminstrative capabilities for data holdings of a server. These items retain important information pertinent to publishing in a Web GIS deployment. See the [Data](/rest/enterprise-administration/server/data/) documentation for full details.

Let's demonstrate using the *DataStoreManager* to add an object store to our GIS. First we'll get the *DataStoreManager*, then construct an object representing the object store, then pass that object to the [add()](/python/latest/api-reference/arcgis.gis.server.html#arcgis.gis.server.DataStoreManager.add) method to regtister the object store with the server.

In [8]:
datastore_mgr = gis_server.datastores

### Construct the Object Store Connection object

We'll use a Python dictionary to create an object that holds all the necessary information about the object store to add it to the server. See [Object store](https://developers.arcgis.com/rest/enterprise-administration/server/registerdataitem/#object-store) and [Object store examples](https://developers.arcgis.com/rest/enterprise-administration/server/registerdataitem/#object-store-examples) for specific details and requirements for creating the object.

In [18]:
object_str_config: dict =  {
    "path": "/cloudStores/api_object_store",
    "type": "objectStore",
    "provider": "amazon",
    "info": {
        "isManaged": True,
        "systemManaged": False,
        "isManagedData": True,
        "purposes": ["feature-tile", "scene"],
        "connectionString": "{\"accessKeyId\":\"<access_key_value>\",\"secretAccessKey\":\"<secret_access_value>\",\"regionEndpointUrl\":\"https://s3.us-west-2.amazonaws.com\",\"defaultEndpointsProtocol\":\"https\",\"credentialType\":\"accesskey\",\"region\":\"us-west-2\"}",
        "objectStore": "s3bucket/1091cache"
    }
}

### Add the Object Store to the server

In [19]:
object_data_store = datastore_mgr.add(
    item=object_str_config
)

In [20]:
object_data_store

<Datastore title:"https://example.site.com:6443/arcgis/admin/data/items/cloudStores/api_object_store" type:"objectStore">

## Validate the Data Store

In [21]:
object_data_store.validate()

KeyError: 'status'

## Updating the Configuration

The validation failed. There may be something in the object store construction that is causing the failure. Let's update the data store configuration.

### Get the Configuration
The *properties* property on the object store returns the configuration.

In [22]:
obj_store_config = object_data_store.properties
obj_store_config

{
  "path": "/cloudStores/api_object_store",
  "type": "objectStore",
  "id": "2eaad7a5-3be6-47ef-bd78-ae0e26788d53",
  "systemManaged": false,
  "provider": "amazon",
  "info": {
    "isManaged": true,
    "systemManaged": false,
    "isManagedData": true,
    "purposes": [
      "feature-tile",
      "scene"
    ],
    "connectionString": "{crypt}B3R2/3FKfR45nMTX/Xcoj3AE5V9N2JXtYKe4DyxN0GhZOjYr6sNUWh2JBOOXh2eayeidble/AlEVlJcUhZRNGubB9VqoE4av8/BidobKtO11Fxtzt2UJqn1Q7v7uRJbHRB7drioDhNfYil0KCrNCqbtNFR4awURk8gX2tEy4GYDqXnxKkpwCi/qDr/8Xm8azV0K2NnQrlFcVsw0BT0O+2u/w8Q7GM5ZZjKx1r/rUEv1McgeLydaVveIw8WLxd+n/eboX/6Pab3Bj4HAmYba5N14/mJoerQSHmNFwFdkNCv4Yy2u/QHJQMzAhXJPazDSN4WJyzirWd9k=",
    "objectStore": "s3bucket/1091cache"
  }
}

###  Update the configuration information

First, update the properties of the dictionary that represents the object data store connection information.

In [33]:
obj_store_config.update({
    "path": "/cloudStores/api_object_store",
    "type": "objectStore",
    "provider": "amazon",
    "info": {
        "isManaged": True,
        "systemManaged": False,
        "isManagedData": True,
        "purposes": ["feature-tile", "scene"],
        "connectionString": "{\"accessKeyId\":\"<access_key_value>\",\"secretAccessKey\":\"<secret_access_value>\",\"regionEndpointUrl\":\"https://s3.us-west-2.amazonaws.com\",\"defaultEndpointsProtocol\":\"https\",\"credentialType\":\"accesskey\",\"region\":\"us-west-2\"}",
        "objectStore": "1091cache",
    }
})

### Update the data store configuration on the server
Use the [*update()*](/python/latest/api-reference/arcgis.gis.server.html#arcgis.gis.server.Datastore.update) method to edit the data store connection information.

In [30]:
object_data_store.update(obj_store_config)

True

### Validate the updated Data Store

Run the validation again, and then confirm the properties are valid.

In [31]:
object_data_store.validate()

True

In [32]:
object_data_store.properties

{
  "path": "/cloudStores/api_object_store",
  "type": "objectStore",
  "id": "2eaad7a5-3be6-47ef-bd78-ae0e26788d53",
  "systemManaged": false,
  "provider": "amazon",
  "info": {
    "isManaged": true,
    "systemManaged": false,
    "isManagedData": true,
    "purposes": [
      "feature-tile",
      "scene"
    ],
    "connectionString": "{\"accessKeyId\":\"<access_key_value>\",\"secretAccessKey\":\"<secret_access_value>\",\"regionEndpointUrl\":\"https://s3.us-west-2.amazonaws.com\",\"defaultEndpointsProtocol\":\"https\",\"credentialType\":\"accesskey\",\"region\":\"us-west-2\"}",
    "objectStore": "1091cache"
  }
}